In [163]:
import pandas as pd
import datetime as dt 
from datetime import date
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

nmbrZones = 2

co = pd.read_csv(r'./TQ5-06-12-2021-Data.csv')
co = co.append(pd.read_csv(r'./TQ5-07-12-2021-Data.csv'))

co['DateTime'] = pd.to_datetime(co.Date + ' ' + co.Time)
co['hour'] = co.DateTime.dt.hour
co

C:\Users\17867\AppData\Local\Temp/ipykernel_19244/3951010495.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\17867\AppData\Local\Temp/ipykernel_19244/3951010495.py:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


,Date,Time,Zone,Device,PPM,VOC,DateTime,hour
0,12/06/2021,11:42,Zone1,T1,442,6,2021-12-06 11:42:00,11
1,12/06/2021,11:42,Zone1,T2,455,8,2021-12-06 11:42:00,11
2,12/06/2021,11:44,Zone1,T1,475,11,2021-12-06 11:44:00,11
3,12/06/2021,11:44,Zone1,T2,478,11,2021-12-06 11:44:00,11
4,12/06/2021,11:44,Zone2,T1,1453,160,2021-12-06 11:44:00,11
...,...,...,...,...,...,...,...,...
733,12/07/2021,16:24,Zone2,T2,436,5,2021-12-07 16:24:00,16
734,12/07/2021,16:25,Zone1,T1,459,8,2021-12-07 16:25:00,16
735,12/07/2021,16:25,Zone1,T2,434,5,2021-12-07 16:25:00,16
736,12/07/2021,16:25,Zone2,T1,422,3,2021-12-07 16:25:00,16


In [14]:
# Displaying data per sensor
d = {'PPM': [], 'VOC': []} 

today = co.Date.unique()

date_list = [i for i in today]
time_list = [str(i) for i in range(8,18)]
zone_list = ["Zone"+str(i) for i in range(1,nmbrZones+1)]

dataframe_collection = {} 

for zone in zone_list:
    dataframe_collection[zone] = pd.DataFrame()

for key in dataframe_collection.keys():
    dataframe_collection[key] = co.groupby(by=['Zone']).get_group(key)
    
grouped = co.groupby(["Zone","Date","hour"])

for key,item in grouped:
  a_group = grouped.get_group(key)
  print(a_group, "\n")
  #print(key, "\n")


          Date   Time   Zone Device   PPM  VOC            DateTime  hour
0   12/06/2021  11:42  Zone1     T1   442    6 2021-12-06 11:42:00    11
1   12/06/2021  11:42  Zone1     T2   455    8 2021-12-06 11:42:00    11
2   12/06/2021  11:44  Zone1     T1   475   11 2021-12-06 11:44:00    11
3   12/06/2021  11:44  Zone1     T2   478   11 2021-12-06 11:44:00    11
6   12/06/2021  11:45  Zone1     T1   523   18 2021-12-06 11:45:00    11
7   12/06/2021  11:45  Zone1     T2   448    7 2021-12-06 11:45:00    11
10  12/06/2021  11:47  Zone1     T1   455    8 2021-12-06 11:47:00    11
11  12/06/2021  11:47  Zone1     T2   455    8 2021-12-06 11:47:00    11
14  12/06/2021  11:48  Zone1     T1   433    5 2021-12-06 11:48:00    11
15  12/06/2021  11:48  Zone1     T2   473   11 2021-12-06 11:48:00    11
16  12/06/2021  11:49  Zone1     T1   496   14 2021-12-06 11:49:00    11
17  12/06/2021  11:49  Zone1     T2   477   11 2021-12-06 11:49:00    11
18  12/06/2021  11:50  Zone1     T1   441    6 2021

In [13]:
app = JupyterDash(__name__)

app.layout = html.Div(children=[
    html.H1(children='AirQuino Dashboard'),
    dcc.Tabs(id="tabs-graph", value='tab-1-line-graph', children=[
        dcc.Tab(label='Line Graph', value='tab-1-line-graph'),
        dcc.Tab(label='Heat Map', value='tab-2-heat-map'),
    ]),
    html.Div([
        html.Div([dcc.Dropdown(id='zone-dropdown', 
                               clearable=False,
                               value='Zone1',
                               options=[{'label': zone, 'value': zone}for zone in zone_list])],
                 style={'width': '40%', 'float': 'left'}),
        html.Div([dcc.Dropdown(id='date-dropdown', 
                               clearable=False,
                               value=date_list[0],
                               options=[{'label': day, 'value': day}for day in date_list])],
                 style={'width': '40%', 'float': 'left'})
    ], style={'width': '100%'}),
    
    html.Div(id='tabs-content-example-graph', style={'position': 'absolute', 'bottom': '230px', 'width': '90%'}),

])

# Define callback to update graph
@app.callback(
    Output('tabs-content-example-graph', 'children'),
    Input("tabs-graph", "value"), 
    Input("zone-dropdown", "value"),
    Input("date-dropdown", "value")
)


def render_content(tab, zone, date):
    if(tab == 'tab-1-line-graph'):
        df = dataframe_collection[zone].groupby(by=['Date']).get_group(date)
        line = px.line(df, x='Time', y='VOC', color='Device')
        return html.Div([dcc.Graph(id='graph-1-tabs',
                                   figure= line)])
    elif(tab == 'tab-2-heat-map'):
        heat = px.density_heatmap(dataframe_collection[zone].groupby('Date').get_group(date), x='Date', y='Time', z='VOC', facet_col="Device", color_continuous_scale="YlOrRd")
        return html.Div([dcc.Graph(id='graph-2-tabs',
                                   figure=heat)])
 

app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


D:\Semester 4\Advanced Phase\Data\Mini Projects\AirQuino\Dashboard\env\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

